In [1]:
!cd container; ./build_and_push.sh gnn-inference

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  46.08kB
Step 1/16 : ARG REGION=us-west-2
Step 2/16 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

a72a686c: Pulling fs layer 
9a5544da: Pulling fs layer 
7c550755: Pulling fs layer 
Digest: sha256:3f3ee50cb89bc12028bab7d1e187ae57f12b957135b91648702e835c37c6c971
Status: Downloaded newer image for ubuntu:16.04
 ---> 96da9143fb18
Step 3/16 : LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
 ---> Running in 3cc578bcb23f
Removing intermediate container 3cc578bcb23f
 ---> a913449a01cb
Step 4/16 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Running in cb9cf56b61b9
Removing intermediate containe

In [2]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

# bucket = 'sagemaker-{}-{}'.format(region, account_id)
# prefix = 'demo-multimodel-endpoint'

role = get_execution_role()

In [3]:
# import models into hosting

from time import gmtime, strftime

model_name = 'DEMO-MultiModelModel' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
# model_url = 'https://s3-{}.amazonaws.com/{}/{}/'.format(region, bucket, prefix)
model_url = 's3://jdurago-insight-2020a/output/baseline/drug-prediction-gcn-200128-2135-015-1feea50f/output/'
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'gnn-inference')

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: DEMO-MultiModelModel2020-02-02-19-41-05
Model data Url: s3://jdurago-insight-2020a/output/baseline/drug-prediction-gcn-200128-2135-015-1feea50f/output/
Container image: 405787280505.dkr.ecr.us-west-2.amazonaws.com/gnn-inference:latest
Model Arn: arn:aws:sagemaker:us-west-2:405787280505:model/demo-multimodelmodel2020-02-02-19-41-05


In [4]:
# create endpoint configuration

endpoint_config_name = 'DEMO-MultiModelEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: DEMO-MultiModelEndpointConfig-2020-02-02-19-41-05
Endpoint config Arn: arn:aws:sagemaker:us-west-2:405787280505:endpoint-config/demo-multimodelendpointconfig-2020-02-02-19-41-05


In [ ]:
# create endpoint
import time

endpoint_name = 'DEMO-MultiModelEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: DEMO-MultiModelEndpoint-2020-02-02-19-41-08
Endpoint Arn: arn:aws:sagemaker:us-west-2:405787280505:endpoint/demo-multimodelendpoint-2020-02-02-19-41-08
Endpoint Status: Creating
Waiting for DEMO-MultiModelEndpoint-2020-02-02-19-41-08 endpoint to be in service...


# Invoke Models

In [ ]:
payload = [{'body': [1,2]}]
payload = json.dumps(payload)

In [66]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    TargetModel='model.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=payload)

print(*json.loads(response['Body'].read()), sep = '\n')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (503) from model with message "{
  "code": 503,
  "type": "InternalServerException",
  "message": "Prediction failed"
}
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/DEMO-MultiModelEndpoint-2020-02-02-01-23-29 in account 405787280505 for more information.

In [33]:
payload

'[{"body": [1, 2]}]'